# Import libraries

In [28]:
from __future__ import division
from sklearn.externals import joblib
#import joblib
import glob
import pickle
from sklearn.preprocessing import MinMaxScaler
import scipy.io as sio
import numpy as np
import ntpath
import collections
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
from PIL import ImageOps
#from skimage.color import rgb2hed, rgb2gray

In [37]:
# Load the normalization scaler
scaler = joblib.load('/pickle/scale/clusterLCS_Scale.pickle')
# Load the trained module
dpgmm = joblib.load('/pickle/clusterLCS_8.pickle')
# Path for saving the clustered images
pathtest_save = '/output/phenoTIL/cluster/'
pathtest_set = '/output/*/'
# Load the IF set
test_files = glob.glob(pathtest_set)


In [38]:
for x,xx in zip(test_files, range(0,len(test_files))):
    test_mat = glob.glob(x + '*.mat')
    pathtobesaved = pathtest_save + os.path.split(os.path.dirname(test_mat[0]))[1] + '/'
    if not os.path.exists(pathtobesaved):
        os.mkdir(pathtobesaved)
    for y,yy in zip(test_mat,range(0,len(test_mat))):
        # Name of the folder
        # Name of the cluster model and directory
        curr_image = ntpath.basename(y)[0:-4]
        if not os.path.isfile(pathtobesaved+'/'+curr_image+'.mat'):
            get_siz = os.path.getsize(y)
            if round(get_siz/1024) < 101:
                continue
            print('Loading image {}'.format(curr_image))
            mat_T = sio.loadmat(y)
            cent_T = mat_T['centroids_L']
            L_T = mat_T['localFeatures_L']
            C_T = mat_T['contextFeatures_L']
            S_T = mat_T['graphInterplayFeatures']
            feat_T = np.hstack((L_T,C_T,S_T))
            naNs_val = np.isnan(feat_T)
            feat_T[naNs_val] = 0
            infs_val = np.isinf(feat_T)
            feat_T[infs_val] = 0
            print(np.shape(feat_T))
            M_norm = scaler.transform(feat_T)
            T_X = dpgmm.predict(M_norm)
            fildmp = {'c_features':feat_T, 'c_centroids':cent_T, 'cluspred':T_X}
            sio.savemat(pathtobesaved+'/'+curr_image+'.mat', {'c_features':feat_T, 'c_centroids':cent_T, 'cluspred':T_X})
        else:
            continue
print('all done')

all done


In [ ]:
mat_T = sio.loadmat(y)

In [ ]:
print(mat_T)

In [ ]:
print(round(get_siz/1024))